In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [43]:
from mvl import genData
from torch import tensor

liabParams55cov = genData.genParams(pis=tensor([.1, .1, .05]), rrMeans=tensor([3., 2.]), afMean = tensor(1e-4), pDs = tensor([.01, .01]), afShape=tensor(50.), nCases=tensor([1.5e4, 1.5e4, 4e3]), nCtrls=tensor(5e4), covShared=tensor([ [1, .95], [.95, 1] ]), meanEffectCovarianceScale=tensor(.01))[0]
liabParams55cov["pDs"] = liabParams55cov["pDs"][0:2]
liabData55cov = genData.v6liability(**liabParams55cov)
liabParams55cov["pDs"] = liabData55cov["PDs"] #TODO: normalized names, and indicate that the params pDs are incomplete

covShared tensor([[1.0000, 0.9500],
        [0.9500, 1.0000]])
residualCovariance tensor([[1, 0],
        [0, 1]])
PD1 threshold, PD2 threshold tensor(-2.3263) tensor(-2.3263)
pDsWithBoth tensor([0.0100, 0.0100, 0.0067], dtype=torch.float64)
pdThresh tensor([2.3263, 2.3263])
pdTarget tensor([0.0300, 0.0200])
pdvthresh tensor([1.8808, 2.0537])
meanEffect tensor([0.4456, 0.2726])
meanEffectsAcrossAllGenes tensor([0.4456, 0.2726])
allEffects tensor([[-0.3891, -0.2083],
        [-0.5965, -0.4642],
        [-0.3894, -0.2065],
        ...,
        [-0.4650, -0.3077],
        [-0.4061, -0.1636],
        [-0.5730, -0.3954]])
PD1GivenV.mean() tensor(0.0307) PD2GivenV.mean() tensor(0.0205)
allEffects[i] tensor([-0.3891, -0.2083])
PDBothGivenV.mean tensor(0.0171, dtype=torch.float64)
PDBothGivenV / PDBoth tensor(2.5597, dtype=torch.float64)
pdsCovarOnMean.mean(0) tensor([0.0307, 0.0205, 0.0171], dtype=torch.float64)
np.corrcoef(pdvInBoth[:,0], pdvInBoth[:,1])
 [[1.         0.94908539]
 [0.9490853

In [3]:
liabParams55cov["pDs"]

tensor([0.0100, 0.0100, 0.0067], dtype=torch.float64)

In [4]:
liabParams55cov["pDsample"]

KeyError: 'pDsample'

In [125]:
import numpyro
from numpyro.distributions import Multinomial, Beta, Dirichlet, Gamma, Beta, Categorical, Uniform, MultivariateNormal, Normal, LogNormal, Exponential
from jax import random
from numpyro.infer import MCMC, NUTS
import numpy as np
import jax.numpy as jnp

numpyro.set_host_device_count(6)
numpyro.enable_x64()

def mix_weights(beta: jnp.array):
    beta_cumprod = (1 - beta).cumprod(-1)
    return jnp.pad(beta, (0,1), constant_values=1) * jnp.pad(beta_cumprod, (1,0), constant_values = 1)


# mu_exp, var_exp = get_log_params(liabParams55cov["afMean"].numpy(), 1)

nHypotheses = 6
kConditions = 4
altCounts = liabData55cov["altCounts"].numpy()
N = len(liabData55cov["altCounts"])

nCases = liabParams55cov["nCases"].numpy()
nCtrls = liabParams55cov["nCtrls"].numpy()
samplePDs = nCases / (nCases.sum() + nCtrls)
pdsAll = np.array([1 - samplePDs.sum(), *samplePDs])
print("pdsAll", pdsAll)

# TODO: do this in numpy natively
pdsAllShaped = jnp.asarray(tensor(pdsAll).expand(nHypotheses, kConditions).numpy())
pdsAllnp = [1 - samplePDs.sum(), *samplePDs]
pdsAll = jnp.asarray([1 - samplePDs.sum(), *samplePDs])
print(pdsAllShaped)
print(pdsAll.shape)
print((1/pdsAll).shape)
alpha = .01
print("nHypotheses", nHypotheses)

component_af_means = jnp.array([.005, .006, .006, .013])
exponential_prior = jnp.array([.001]).repeat(kConditions)
# exponential_prior = jnp.array([[.001]]).repeat((kConditions, nHypotheses))
# print("exponential_prior", exponential_prior)

def model(data):
    with numpyro.plate("conc_plate", kConditions):
        conc = numpyro.sample('conc', Exponential(.001))

    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("prob_plate", nHypotheses):
        probs = numpyro.sample("probs", Dirichlet(conc))

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]*pdsAll), obs=data)

def modelLogNormal(data):
    with numpyro.plate("conc_plate", kConditions):
        conc = numpyro.sample('conc', Exponential(.001))

    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("prob_plate", nHypotheses):
        probs = numpyro.sample("probs", Dirichlet(conc))

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]*pdsAll), obs=data)
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

def modelPDScaledDeterministic(data):
    pdsSite = numpyro.deterministic("pDs", pdsAll)
    # mu = numpyro.sample('mu', Uniform(jnp.ones(kConditions), 1000))
    # print("mu", mu)
    # concentrations = numpyro.sample("concentrations", Uniform(1, 1000))
    # print("concentrations", concentrations)
    with numpyro.plate("beta_plate2", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("mu_plate", nHypotheses):
        mu = numpyro.sample('mu', Uniform(jnp.ones(kConditions), 1000))
        probs = numpyro.sample("probs", Dirichlet(mu))
        # print("probs", probs)
        probs = numpyro.deterministic("probs_scaled", probs * pdsAll)
        # print("probs after scaling", probs)

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)


def modelAf(data):
    # pds = numpyro.deterministic("pds", liabParams55cov["afMean"].numpy())# numpyro.sample("af", LogNormal(mu_exp, var_exp)
    # print("af", af)
    # mu = numpyro.sample('mu', Uniform(jnp.ones(kConditions), 1000))
    # print("mu", mu)
    # concentrations = numpyro.sample("concentrations", Uniform(1, 1000))
    # print("concentrations", concentrations)
    with numpyro.plate("beta_plate2", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("mu_plate", nHypotheses):
        conc = numpyro.sample('conc', Uniform(jnp.ones(kConditions), 1000))
        # doing this speeds up MCMC by 2x
        pdconc = numpyro.deterministic("pdconc", conc * pdsAll)
        probs = numpyro.sample("probs", Dirichlet(pdconc))
        # print("probs", probs)
        # probs = numpyro.deterministic("probs_scaled", probs * af)
        # print("probs after scaling", probs)

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        probs = probs[z] / component_af_means[z]
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)


# def modelLogNormal(data):
#     # tau = numpyro.param('tau', MultivariateNormal(jnp.zeros(kConditions), 3 * jnp.eye(kConditions)))
#     with numpyro.plate("beta_plate", nHypotheses-1):
#         beta = numpyro.sample("beta", Beta(1, alpha))

#     with numpyro.plate("mu_plate", nHypotheses):
#         # rrMean = numpyro.param("rrMean")
#         rr = numpyro.sample("rr", LogNormal(jnp.eye(kConditions), jnp.eye(kConditions)).to_event(1))
#         af = numpyro.sample("af", LogNormal(jnp.array(liabParams55cov["afMean"]), jnp.ones(kConditions)).to_event(1))
#         pvd = rr * af
#         print("pvd", pvd)
#         pvdpv = pvd*pdsAll
#         print("pvdpv", pvdpv)
        
#         probs = numpyro.sample("probs", Dirichlet(pvdpv))
#         # just slows things down probs = numpyro.deterministic("probs_scaled", probs * pdsAll)

#     with numpyro.plate("data", N):
#         z = numpyro.sample("z", Categorical(mix_weights(beta)))
#         return numpyro.sample("obs", Multinomial(probs=pvdpv[z]), obs=data)

# def modelGamma(data):
#     with numpyro.plate("beta_plate", nHypothesis - 1):
#         beta = numpyro.sample("beta", Beta(1, alpha))

#     with numpyro.plate("mu_plate", nHypotheses):
#         rrMean = numpyro.param("rrMean")
#         rr = numpyro.sample("rr", Gamma(jnp.ones(kConditions), jnp.ones(kConditions)))
#         af = numpyro.sample("af", Gamma(liabParams55cov["afMean"], jnp.ones(kConditions)))
#         pvd = rr * liabParams55cov["afMean"]
#         pvdpv = pvd*pdsAll
        
#         probs = numpyro.sample("probs", Dirichlet(np.ones(kConditions)))
#         # just slows things down probs = numpyro.deterministic("probs_scaled", probs * pdsAll)

#     with numpyro.plate("data", N):
#         z = numpyro.sample("z", Categorical(mix_weights(beta)))
#         return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

pdsAll [0.59523809 0.17857143 0.17857143 0.04761905]
[[0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]]
(4,)
(4,)
nHypotheses 6


In [126]:
np.array([.001]).repeat((4,2))

ValueError: operands could not be broadcast together with shape (1,) (2,)

In [127]:
component_af_means[jnp.array([2,3,0,1,1,1])]

DeviceArray([0.006, 0.013, 0.005, 0.006, 0.006, 0.006], dtype=float32)

In [129]:
# 1/3rd as fast as not having deterministic
# this runs after mcmc fits, not during
def postprocess(arg):
    print("postprocess arg", arg)
    return arg
init_rng_key = random.PRNGKey(12273)
mcmcAf = MCMC(NUTS(model), 100, 1000, jit_model_args=True)
mcmcAf.run(init_rng_key, altCounts)
mcmcAf.print_summary()

sample: 100%|██████████| 1100/1100 [04:04<00:00,  4.51it/s, 127 steps of size 2.99e-02. acc. prob=0.93]

                mean       std    median      5.0%     95.0%     n_eff     r_hat
   beta[0]      0.10      0.03      0.10      0.06      0.15     82.90      1.02
   beta[1]      0.12      0.04      0.12      0.06      0.18     13.57      1.05
   beta[2]      0.90      0.04      0.90      0.84      0.97     50.15      1.00
   beta[3]      0.96      0.13      1.00      0.91      1.00    108.03      1.00
   beta[4]      0.94      0.16      1.00      0.83      1.00    810.88      1.00
   conc[0]     18.25      6.97     17.18      6.10     27.63    503.21      1.00
   conc[1]     10.13      3.92      9.45      3.89     16.16    492.12      1.00
   conc[2]      8.82      3.35      8.46      3.46     13.85    415.62      1.00
   conc[3]      3.60      1.39      3.41      1.46      5.87    574.65      1.00
probs[0,0]      0.44      0.03      0.45      0.39      0.48      8.21      1.16
prob

In [130]:
# Scaling by pDs gives absolutely no difference in concentrations or probabilities.
sAf = mcmcAf.get_samples()["probs"]
print(sAf.mean(0))

# concs = mcmcAf.get_samples()["conc"]
# print("conc std", concs.std(0))
# print("conc mean", concs.mean(0))
# print("DIrichlet version", Dirichlet(concs.mean(0)).mean)

betaScaledInDirichlet = mcmcAf.get_samples()['beta']
print("inferred stick-breaking weights", mix_weights(betaScaledInDirichlet).mean(0))

[[0.4398996  0.19456017 0.27839905 0.08714116]
 [0.37654528 0.36484608 0.16515735 0.09345128]
 [0.59867483 0.17613707 0.1781984  0.04698981]
 [0.3738497  0.29160115 0.25154197 0.08300716]
 [0.44515297 0.25354034 0.21096157 0.09034516]
 [0.44809085 0.24717084 0.2149813  0.08975691]]
inferred stick-breaking weights [0.10477007 0.10960675 0.7080873  0.07608418 0.00531045 0.00123203]


In [123]:
# Scaling by pDs gives absolutely no difference in concentrations or probabilities.
sAf = mcmcAf.get_samples()["probs"]
print(sAf.mean(0))

# concs = mcmcAf.get_samples()["conc"]
# print("conc std", concs.std(0))
# print("conc mean", concs.mean(0))
# print("DIrichlet version", Dirichlet(concs.mean(0)).mean)

betaScaledInDirichlet = mcmcAf.get_samples()['beta']
print("inferred stick-breaking weights", mix_weights(betaScaledInDirichlet).mean(0))

[[0.39795864 0.38242096 0.11334935 0.10627104]
 [0.34449634 0.34402916 0.23187748 0.07959706]
 [0.5973565  0.176043   0.17932664 0.04727379]
 [0.44580635 0.12760615 0.342677   0.08391044]
 [0.2525074  0.30790552 0.2199237  0.21966338]
 [0.24689339 0.27714565 0.23167437 0.24428658]]
inferred stick-breaking weights [0.10958774 0.08479752 0.72028106 0.08632846 0.00186985 0.0010561 ]


In [131]:
pdv = (liabData55cov["afs"][liabData55cov["unaffectedGenes"]]).mean(0) / liabParams55cov["afMean"]# PD|V
print("pdv", pdv)
print("pdv scaled", pdv/pdv.sum())

pdv tensor([0.5953, 0.1786, 0.1786, 0.0476], dtype=torch.float64)
pdv scaled tensor([0.5952, 0.1786, 0.1786, 0.0476], dtype=torch.float64)


In [132]:
pdv = (liabData55cov["afs"][liabData55cov["affectedGenes"][0]]).mean(0) / liabParams55cov["afMean"]# PD|V
print("pdv", pdv)
print("pdv scaled", pdv/pdv.sum())

pdv tensor([0.5738, 0.5486, 0.1785, 0.1463], dtype=torch.float64)
pdv scaled tensor([0.3965, 0.3791, 0.1233, 0.1011], dtype=torch.float64)


In [133]:
pdv = (liabData55cov["afs"][liabData55cov["affectedGenes"][1]]).mean(0) / liabParams55cov["afMean"]# PD|V
print("pdv", pdv)
print("pdv scaled", pdv/pdv.sum())

pdv tensor([0.5836, 0.1784, 0.3703, 0.0987], dtype=torch.float64)
pdv scaled tensor([0.4741, 0.1449, 0.3008, 0.0802], dtype=torch.float64)


In [134]:
pdv = (liabData55cov["afs"][liabData55cov["affectedGenes"][2]]).mean(0) / liabParams55cov["afMean"]# PD|V
print("pdv", pdv)
print("pdv scaled", pdv/pdv.sum())

pdv tensor([0.5717, 0.5500, 0.3674, 0.1223], dtype=torch.float64)
pdv scaled tensor([0.3548, 0.3413, 0.2280, 0.0759], dtype=torch.float64)


In [79]:
# [[0.01911611 0.01343591 0.01555622 0.00563713]
#  [0.00406165 0.00276914 0.00327851 0.00133072]
#  [0.01090826 0.00992517 0.0113952  0.00424665]
#  [0.01858757 0.02309512 0.02210252 0.01013227]]
# [[0.47631553 0.10743626 0.26105177 0.03134572]
#  [0.60844487 0.14429386 0.14502415 0.01706672]
#  [0.40694135 0.31180432 0.09986749 0.0380971 ]
#  [0.35437214 0.26981634 0.2043957  0.02945131]]
# conc std [ 12.166306  71.86332   67.61688  157.65123 ]
# conc mean [ 36.537464 213.60443  190.46646  793.18304 ]
# DIrichlet version [0.02961397 0.17312849 0.15437493 0.64288265]
# inferred stick-breaking weights [0.11452598 0.665086   0.13667543 0.08672265]

In [80]:
pdvpv = (liabData55cov["afs"][liabData55cov["affectedGenes"][2]]).mean(0) #/ liabParams55cov["afMean"]# PD|V
pdv = pdvpv / liabParams55cov["afMean"]
print("pdv", pdv)
print("pdvpv", pdvpv)
print("pdvpv.sum()", pdvpv.sum())
# This matches our DM results perfectly; so the DM result matches scale
# how to allow our pdv values to work ok...
print("pdvpv scaled", pdvpv/pdvpv.sum())


pdv tensor([0.5717, 0.5500, 0.3674, 0.1223], dtype=torch.float64)
pdvpv tensor([5.7167e-05, 5.5000e-05, 3.6744e-05, 1.2231e-05], dtype=torch.float64)
pdvpv.sum() tensor(0.0002, dtype=torch.float64)
pdvpv scaled tensor([0.3548, 0.3413, 0.2280, 0.0759], dtype=torch.float64)


In [81]:
liabData55cov["altCounts"][liabData55cov["affectedGenes"][2]]

tensor([[4., 3., 3., 0.],
        [8., 3., 0., 1.],
        [6., 3., 2., 0.],
        ...,
        [5., 8., 1., 1.],
        [3., 3., 5., 2.],
        [6., 5., 2., 0.]], dtype=torch.float64)

In [82]:
pdv.sum()

tensor(1.6114, dtype=torch.float64)

In [83]:
(liabData55cov["afs"][liabData55cov["affectedGenes"][2]]).mean(0) 

tensor([5.7167e-05, 5.5000e-05, 3.6744e-05, 1.2231e-05], dtype=torch.float64)

In [84]:
liabData55cov["afs"][liabData55cov["affectedGenes"][0]].mean(0) 

tensor([5.7375e-05, 5.4857e-05, 1.7848e-05, 1.4628e-05], dtype=torch.float64)

In [85]:
pdsAllnp

[0.5952380895614624, 0.17857143, 0.17857143, 0.04761905]

In [86]:
liabParams55cov["afMean"]

tensor(1.0000e-04)

In [87]:
liabData55cov["afs"][liabData55cov["affectedGenes"][0]].mean(0)

tensor([5.7375e-05, 5.4857e-05, 1.7848e-05, 1.4628e-05], dtype=torch.float64)

In [88]:
liabParams55cov["afMean"]

tensor(1.0000e-04)

In [ ]:
# (1000, 4, 4)
# [[0.00514663 0.00313207 0.00355292 0.00149964]
#  [0.01928454 0.04950465 0.04956334 0.0065191 ]
#  [0.02503735 0.04772532 0.03840724 0.00905628]
#  [0.06047119 0.05520838 0.05940869 0.03415463]]
# [[0.607701   0.1456876  0.14313476 0.01758165]
#  [0.45891437 0.14094748 0.24490298 0.03002501]
#  [0.45052725 0.23326729 0.15602364 0.03166293]
#  [0.4925649  0.1685785  0.18608962 0.03147438]]
# conc std [18.310623   7.338482   7.672899   2.7556536]
# conc mean [46.069946 18.652845 19.728569  6.812487]
# DIrichlet version [0.50479954 0.20438373 0.2161707  0.07464606]
# inferred stick-breaking weights [0.6861298  0.14098327 0.1566307  0.01916244]